In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import time
import datetime
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
trainData = pd.read_csv('../input/train.csv', low_memory=False)
trainData

In [ ]:
supply = pd.read_csv('../input/store.csv')
supply.set_index('Store', inplace=True)
supply

In [ ]:
def nanToMedian(df):
    for col in df.columns:
        array = np.array(df[col])
        datetype = type(array[1])
        if(datetype == float or datetype == np.float64):
            array[np.isnan(array)] = np.median(array[~np.isnan(array)])
        df[col] = array

In [ ]:
#заменим NaN во всех столбцах, кроме последнего на медиану столбца
nanToMedian(supply)
supply

In [ ]:
toUnixTime = lambda x: time.mktime(datetime.datetime.strptime(x, '%Y-%m-%d').timetuple())

In [ ]:
trainData['UnixTime'] = trainData['Date'].map(toUnixTime)
trainData.drop('Date', axis=1, inplace=True)

In [ ]:
#отнормируем столбца с Unix временем и заменим буквы на числа в качественных признаках
trainData.StateHoliday.replace({'a': 1, 'b': 2, 'c': 3, '0': 0}, inplace=True)
unixTime = trainData['UnixTime'].values
trainData['UnixTime'] = (unixTime - np.mean(unixTime))/np.std(unixTime)

In [ ]:
trainData

In [ ]:
#Идея: будем обучать 2 модели случайного леса. Одну с целевой переменной customers, вторую с целевой sales
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve

In [ ]:
X_train, customers_train, sales_train = trainData.drop(['Sales', 'Customers'], axis=1).values, trainData['Customers'].values, trainData['Sales'].values

In [ ]:
X1_train = np.concatenate((X_train, np.reshape(customers_train, (customers_train.shape[0], 1))), axis=1)

In [ ]:
#подберем оптимальный параметр глубины первого дерева
axisX = np.linspace(20, 45, num=25).astype(int)
train_scores, test_scores = validation_curve(DecisionTreeRegressor(), X_train, customers_train, 'max_depth', axisX, n_jobs=-1,
                                            scoring='neg_mean_squared_error')

plt.figure()
plt.plot(axisX, np.mean(train_scores, axis=1), 'o-', color='r')
plt.plot(axisX, np.mean(test_scores, axis=1), 'o-', color='b')
plt.xlabel('depth')
plt.ylabel('score')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
forest1 = RandomForestRegressor(n_estimators=80, max_depth=35)
forest2 = RandomForestRegressor()
forest1.fit(X_train, customers_train)
forest2.fit(X1_train, sales_train)

In [ ]:
#загружаем тестовые данные и приводим к тому же виду
testData = pd.read_csv('../input/test.csv')

In [ ]:
testData['UnixTime'] = testData['Date'].map(toUnixTime)
testData.drop('Date', axis=1, inplace=True)

In [ ]:
testData.StateHoliday.replace({'a': 1, 'b': 2, 'c': 3, '0': 0}, inplace=True)
unixTime = testData['UnixTime'].values
testData['UnixTime'] = (unixTime - np.mean(unixTime))/np.std(unixTime)

In [ ]:
testData

In [ ]:
nanToMedian(testData)
X_test = testData.drop('Id', axis=1).values

In [ ]:
customers_test = forest1.predict(X_test)
X1_test = np.concatenate((X_test, np.reshape(customers_test, (customers_test.shape[0], 1))), axis=1)
sales_predicted = forest2.predict(X1_test)

In [ ]:
Result = pd.DataFrame(testData.Id)
Result['Sales'] = sales_predicted
Result.set_index('Id')
Result.to_csv('./result.csv', index=False)